#+NAME: LeNet
#+YEAR: 1985
#+AUTHORS: Yann LeCun, Bernhard Boser, John Denker, Donnie Henderson, Richard Howard, Wayne Hubbard, Lawrence Jackel
#+CATEGORIES: Convolution
#+DESCRIPTION: LeNet is the earliest introduction of the modern convolutional architecture. It contains three alternating convolutional and pooling layers followed by two fully connected layers. This model was introduced alongside the back propagation algorithm which produced incredible results at the time for image classification.
#+PAPER: /static/boltzmann.pdf
#+IMAGE:

# Overview

LeNet is the earliest introduction of the modern convolutional architecture. It contains two alternating convolutional and pooling layers followed by three fully connected layers. This model was introduced alongside the back propagation algorithm which produced incredible results at the time for image classification.

In [ ]:
use burn::{
  config::Config,
  module::Module,
  nn::{
    conv::{Conv2d, Conv2dConfig},
    pool::{AdaptiveAvgPool2d, AdaptiveAvgPool2dConfig},
    Linear, LinearConfig, ReLU,
  },
  tensor::{backend::Backend, Tensor},
};

#[derive(Module, Debug)]
pub struct Model<B: Backend> {
  conv1: Conv2d<B>,
  conv2: Conv2d<B>,
  pool1: AdaptiveAvgPool2d,
  pool2: AdaptiveAvgPool2d,
  linear1: Linear<B>,
  linear2: Linear<B>,
  linear3: Linear<B>,
  activation: ReLU,
}

#[derive(Config, Debug)]
pub struct ModelConfig {
  num_classes: usize,
}

impl ModelConfig {
  pub fn init<B: Backend>(&self, device: &B::Device) -> Model<B> {
    Model {
      conv1: Conv2dConfig::new([1, 6], [5, 5]).init(device),
      conv2: Conv2dConfig::new([6, 16], [5, 5]).init(device),
      pool1: AdaptiveAvgPool2dConfig::new([14, 14]).init(),
      pool2: AdaptiveAvgPool2dConfig::new([5, 5]).init(),
      linear1: LinearConfig::new(16*5*5, 120).init(device),
      linear2: LinearConfig::new(120, 84).init(device),
      linear3: LinearConfig::new(84, self.num_classes).init(device),
      activation: ReLU::new(),
    }
  }

  pub fn init_with<B: Backend>(&self, record: ModelRecord<B>) -> Model<B> {
    Model {
      conv1: Conv2dConfig::new([1, 6], [5, 5]).init_with(record.conv1),
      conv2: Conv2dConfig::new([6, 16], [5, 5]).init_with(record.conv2),
      pool1: AdaptiveAvgPool2dConfig::new([14, 14]).init(),
      pool2: AdaptiveAvgPool2dConfig::new([5, 5]).init(),
      linear1: LinearConfig::new(16*5*5, 120).init_with(record.linear1),
      linear2: LinearConfig::new(120, 84).init_with(record.linear2),
      linear3: LinearConfig::new(84, self.num_classes).init_with(record.linear3),
      activation: ReLU::new(),
    }
  }
}

impl<B: Backend> Model<B> {
  pub fn forward(&self, x: Tensor<B, 4>) -> Tensor<B, 2> {
    let [batch_size, _channels, _height, _width] = x.dims();

    let x = self.conv1.forward(x);
    let x = self.activation.forward(x);
    let x = self.pool1.forward(x);

    let x = self.conv2.forward(x);
    let x = self.activation.forward(x);
    let x = self.pool2.forward(x);

    let x = x.reshape([batch_size, 16*5*5]);
    let x = self.linear1.forward(x);
    let x = self.activation.forward(x);

    let x = self.linear2.forward(x);
    let x = self.activation.forward(x);

    self.linear3.forward(x)
  }
}